### **Peer Collaborative Learning for Online Knowledge Distillation**

#### **Abstract**

- *KD* 依赖 *teacher model*, *collaborative learning* 和 *mutual learning* 没有构建 *online high-capacity teacher model*, 而 *online emsemble* 忽略了分支与其 *logit* 求和之间的协作，阻碍了集成教师的进一步优化
- 本文提出了一种框架(PCL)，将 *online ensembling* 和 *network collaboration* 融合成一个统一的框架
- *Specifically, given a target network, we construct a multi-branch network for training, in which each branch is called a **peer**. We **perform random augmentation** multiple times on the inputs to peers and **assemble feature representations outputted from peers with an additional classifier as the peer ensemble teacher***
- 数据集：*CIFAR-10, CIFAR-100 and ImageNet*

> *Meanwhile, we employ the **temporal mean model of each peer as the peer mean teacher** to collaboratively transfer knowledge among peers, which helps each peer to learn richer knowledge and facilitates to optimise a more stable model with better generalisation*

----------------------------------------------
#### **Introduction**

![Figure 1: Comparing four online knowledge distillation mechanisms](../img/Figure-1_Comparing-four-online-knowledge-distillation-mechanisms.png)

- 对于每个 *peer*， 设计两个 *teacher*
    - *peer ensemble teacher*: 集成的时候使用 *peer* 的 *logit*，**进行 ***concat***，然后使用一个 *classifier* 进行分类**
    - *peer mean teacher: to collaboratively distil knowledge among peers. Instead of directly distilling knowledge among peers using mutual learning, we utilise the **temporal mean model of each peer** to construct the peer mean teacher.*

- 在 *input* 进行多次的 *random augmentation*
- *In testing, we use a **temporal mean model of a peer** for deployment.*

-----------------------------------------------
#### **Related Work**

> *The recent trend in neural network ensembling focuses on **training a single model** and exploiting different **training phases** of a model as an ensemble*

-------------------------------
#### **Peer Collaborative Learning**

![Figure 2: An overview of Peer Collaborative Learning (PCL) for online knowledge distillation.](../img/Figure-2_An-overview-of-Peer-Collaborative-Learning_PCL-for-online-knowledge-distillation.png)

- *As shown in Fig. 2, to facilitate online knowledge distillation, we assemble the feature representation of peers with an additional classifier as **the peer ensemble teacher** and use the temporal mean model of each peer as **the peer mean teacher**.*

- *The training optimisation objective of **PCL contains three components**:*
  - *The **first component** is the standard cross-entropy loss for classification of the peers ($\mathcal{L}^p_{ce}$) and the peer ensemble teacher $\mathcal{L}^t_{ce}$;*
  - *The **second component** is the peer ensemble teacher distillation loss $\mathcal{L}_{pe}$ for transferring knowledge from a stronger teacher to a student, which in turn further improves the ensemble teacher;*
  - *The **third component** is the peer mean teacher distillation loss $\mathcal{L}_{pm}$ for collaboratively distilling knowledge among peers. Thus, the overall objective $\mathcal{L}$ is formulated as:*
$$\mathcal{L} = \mathcal{L}^p_{ce} + \mathcal{L}^t_{ce} + \mathcal{L}_{pe} + \mathcal{L}_{pm}$$

*In **testing, we use a temporal mean model** of a peer for deployment, which has the same number of parameters as the backbone network, so there is no extra inference cost for deployment. In the scenarios where computational cost is less constrained, **feature representations from peer mean teachers plus the additional classifier** can form an ensemble model for deployment to get better performance.*


> 此处的 *Online Ensembling* 采用的不是 *attention* 的方式，而是采用 *concat* 的方式，然后使用一个 *classifier* 进行分类

~~**二者结合一下会不会效果更好呢？**~~

- $\mathcal{L}_{pe}$ **所占的权重逐步增加**，也就是说一开始不要太听 *teacher* 的话，然后逐步增加听 *teacher* 的话，**详见原文**

- *Peer Mean Teacher* 使用了 *temporal mean model*，也就是说使用了 *exponential moving average*，**详见原文**

##### **Peer Mean Teacher**

- 为了让模型尽量的**多而不同**，这里对于输入 X 使用了 m 次的**随机增强**。之后的 m 个值作为 m 个分支的输入
- 对于 *peer mean teacher*，不需要要是反向传播进行 *update*，而是使用 *exponential moving average* 进行 *update*，**详见原文**
- *peer mean teacher* 能力在刚开始的时候比较弱，用来更新下次的 *peer mean teacher* 时候所占的比重比较小，之后逐渐增大
- 为了稳定，这里的每一个 *peer* 都要和除了自己生成的那个之外的所有的 *peer mean teacher* 进行计算 KL 散度